## Import dependencies

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Initial look at the data

In [2]:
data = pd.read_csv(Path(globals()['_dh'][0]) / "data" / "LanguageDetection.csv", sep=",")
data.sample(5)

,Text,Language
10220,ನಾವು ಚಲನಚಿತ್ರಗಳಿಗೆ ಹೋಗುತ್ತೇವೆ ಎಂದು ನೀವು ಏನು ಹೇ...,Kannada
9794,Sie treffen den Nagel auf den Kopf.,German
7355,L'apprendimento automatico si sviluppa con lo ...,Italian
1404,ഇന്ന് ഇന്റർനെറ്റിൽ ഏറ്റവും പ്രശസ്തമായ പൊതു-അവല...,Malayalam
6418,"еще один способ спросить, не уловил ли вы част...",Russian


In [3]:
print("Number of samples by language:")
data.groupby(by=["Language"]).count()["Text"].sort_values()

Number of samples by language:


Language
Hindi           63
Greek          365
Kannada        369
Danish         428
Tamil          469
German         470
Turkish        474
Arabic         536
Dutch          546
Malayalam      594
Sweedish       676
Russian        692
Italian        698
Portugeese     739
Spanish        819
French        1014
English       1385
Name: Text, dtype: int64

## Train/test split

In [4]:
x = data["Text"].values
y = data["Language"].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
print(x_train.shape)
x_train

(8269,)


array(['τώρα αργότερα η Μέλι και ο Τέρι έσπασαν αντίο στον παλιό τους φίλο και πήγαν να χαμογελούν ο ένας στον άλλο κρυφά εκείνο το βράδυ, τόσο μητέρα όσο και κόρη.',
       'Améliorez-le ou discutez-en.',
       'non ne vale la pena, personalmente amo la frase you rock che significa che sei fantastico.',
       ...,
       'Si alguien te pregunta si estás cansado y quieres decir que no estás cansado en absoluto.',
       'Due to its generality, the field is studied in many other disciplines, such as game theory, control theory, operations research, information theory, simulation-based optimization, multi-agent systems, swarm intelligence, statistics and genetic algorithms.',
       'Sono disponibili inoltre applicazioni dedicate per i vari dispositivi mobili, ad esempio Wikipedia Mobile per iPhone e Wikipedia Mobile per Android.'],
      dtype=object)

## ML pipeline

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



clf_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(token_pattern=r"[\w_]{2,}", lowercase=False)),
    ("naive_bayes", MultinomialNB(alpha=.01)),
])
#clf_pipeline = Pipeline([
#    ("wordcount", CountVectorizer()),
#    ("naive_bayes", MultinomialNB(alpha=.01)),
#])
clf_pipeline.fit(x_train, y_train)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False, token_pattern='[\\w_]{2,}')),
                ('naive_bayes', MultinomialNB(alpha=0.01))])

## Test prediction and sanity check

In [7]:
pred = clf_pipeline.predict(x_test)

print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

      Arabic       1.00      0.98      0.99       106
      Danish       0.99      0.96      0.97        73
       Dutch       0.99      0.98      0.99       111
     English       0.92      1.00      0.96       291
      French       1.00      0.99      0.99       219
      German       1.00      0.98      0.99        93
       Greek       1.00      1.00      1.00        68
       Hindi       1.00      1.00      1.00        10
     Italian       1.00      0.99      0.99       145
     Kannada       1.00      1.00      1.00        66
   Malayalam       1.00      0.98      0.99       121
  Portugeese       0.99      0.97      0.98       144
     Russian       1.00      0.99      0.99       136
     Spanish       0.99      0.97      0.98       160
    Sweedish       0.99      0.98      0.99       133
       Tamil       1.00      0.99      0.99        87
     Turkish       1.00      0.98      0.99       105

    accuracy              

In [35]:
for sample in ["Ein kleiner deutscher Text", "A small text without meaning", "C'è un pò d'italiano", "Une petite histoire de Paris", "Генсек ООН призвал к соблюдению перемирия во время Олимпиады"]:
    print(sample, "=>", clf_pipeline.predict(np.array([sample]))[0], np.max(clf_pipeline.predict_proba(np.array([sample]))[0]))
    clf_pipeline.predict_proba(np.array([sample]))[0]
    print(sample, "=>", clf_pipeline.predict(np.array([sample]))[0], np.max()

Ein kleiner deutscher Text => German 0.768441150069108
A small text without meaning => English 0.9986415618842468
C'è un pò d'italiano => Italian 0.7141641875031932
Une petite histoire de Paris => French 0.9968009803867346
Генсек ООН призвал к соблюдению перемирия во время Олимпиады => Russian 0.9912020536752277


In [38]:
np.argmax(clf_pipeline.predict_proba(np.array(["A small text without meaning"])))

3

In [39]:
clf_pipeline.classes_[3]

'English'

## Serialize model

In [9]:
import pickle

with open("data/classifier.pickle", "wb") as f:
    pickle.dump(clf_pipeline, f)

In [10]:
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType

tfidf_settings = {
    TfidfVectorizer: {
        "tokenexp": r"[\pL\pN_]{2,}"
    }
}
initial_type = [('string_input', StringTensorType([None, 1]))]
onx = convert_sklearn(clf_pipeline, initial_types=initial_type, options=tfidf_settings)
with open("data/classifier.onnx", "wb") as f:
    f.write(onx.SerializeToString())

/home/christian/.cache/pypoetry/virtualenvs/demo-api-td6ufWwZ-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute coef_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/christian/.cache/pypoetry/virtualenvs/demo-api-td6ufWwZ-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute intercept_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/home/christian/.cache/pypoetry/virtualenvs/demo-api-td6ufWwZ-py3.9/lib/python3.9/site-packages/skl2onnx/common/_container.py:603: UserWarning: Unable to find operator 'Tokenizer' in domain 'com.microsoft' in ONNX, op_version is forced to 1.
  warnings.warn(


## Differences in tokenization and normalization:

In [11]:
vectorizer = TfidfVectorizer(token_pattern=r"[İ\w_]{2,}", lowercase=False)
vectorizer.fit(x_train)

TfidfVectorizer(lowercase=False, token_pattern='[İ\\w_]{2,}')

In [12]:
tfidf_settings = {
    TfidfVectorizer: {
        "tokenexp": r"[\pL\pN_]{2,}"
    }
}
initial_type = [('string_input', StringTensorType([None, 1]))]
onx = convert_sklearn(vectorizer, initial_types=initial_type, options=tfidf_settings)

In [14]:
import onnxruntime

with open("data/vectorizer.onnx", "wb") as f:
    f.write(onx.SerializeToString())
session = onnxruntime.InferenceSession("data/vectorizer.onnx")
inputs = {'string_input': x_test[:1]}
pred_onx = session.run(None, {"string_input": np.array(["И с этими словами она села в его карету, и, даже не"]).reshape(1, 1)})

In [15]:
for t in x_train:
    pred_sklearn = np.sum(vectorizer.transform(np.array([t])))
    pred_onx = np.sum(session.run(None, {"string_input": np.array([t]).reshape(1, 1)}))
    if abs(pred_onx - pred_sklearn) > 0.01:
        print(t, pred_sklearn, pred_onx)
        print(sorted(list(vectorizer.inverse_transform(vectorizer.transform(np.array([t])))[0])))
        print(sorted(list(vectorizer.inverse_transform(session.run(None, {"string_input": np.array([t]).reshape(1, 1)})[0])[0])))

## Test prediction with ONNX model

In [24]:
import onnxruntime

session = onnxruntime.InferenceSession("data/classifier.onnx")
pred_onx = session.run(None, {"string_input": np.array(["И с этими словами она села в его карету, и, даже не"]).reshape(1, 1)})
print("predict", pred_onx[0])
print("predict_proba", pred_onx[1])

predict ['Russian']
predict_proba [{'Arabic': 7.943455784698017e-06, 'Danish': 1.0405490684206598e-05, 'Dutch': 8.303594768221956e-06, 'English': 2.679041699593654e-06, 'French': 3.670612613859703e-06, 'German': 1.1265211469435599e-05, 'Greek': 1.0832965926965699e-05, 'Hindi': 1.1893760529346764e-05, 'Italian': 5.484678695211187e-06, 'Kannada': 1.8420292690279894e-05, 'Malayalam': 1.082171183952596e-05, 'Portugeese': 5.10070185555378e-06, 'Russian': 0.9998569488525391, 'Spanish': 5.064754532213556e-06, 'Sweedish': 6.081358151277527e-06, 'Tamil': 1.3298573321662843e-05, 'Turkish': 1.1620059012784623e-05}]


In [34]:
for sample in ["Ein kleiner deutscher Text", "A small text without meaning", "C'è un pò d'italiano", "Une petite histoire de Paris", "Генсек ООН призвал к соблюдению перемирия во время Олимпиады"]:
    pred_onx = session.run(None, {"string_input": np.array([sample]).reshape(1, 1)})
    print(sample, "=>", pred_onx[0], pred_onx[1][0][pred_onx[0][0]])
    #print("predict_proba", pred_onx[1])

Ein kleiner deutscher Text => ['German'] 0.7684409618377686
A small text without meaning => ['English'] 0.9986410140991211
C'è un pò d'italiano => ['Italian'] 0.7141642570495605
Une petite histoire de Paris => ['French'] 0.9968007802963257
Генсек ООН призвал к соблюдению перемирия во время Олимпиады => ['Russian'] 0.991202175617218


0.7684409618377686

In [18]:
pred_onnx = session.run(None, {"string_input": np.array([s.encode("utf-8") for s in x_test]).reshape(len(x_test), 1)})
print(metrics.classification_report(y_test, pred_onnx[0]))


              precision    recall  f1-score   support

      Arabic       1.00      0.98      0.99       106
      Danish       0.99      0.96      0.97        73
       Dutch       0.99      0.98      0.99       111
     English       0.92      1.00      0.96       291
      French       1.00      0.99      0.99       219
      German       1.00      0.98      0.99        93
       Greek       1.00      1.00      1.00        68
       Hindi       1.00      1.00      1.00        10
     Italian       1.00      0.99      0.99       145
     Kannada       1.00      1.00      1.00        66
   Malayalam       1.00      0.98      0.99       121
  Portugeese       0.99      0.97      0.98       144
     Russian       1.00      0.99      0.99       136
     Spanish       0.99      0.97      0.98       160
    Sweedish       0.99      0.98      0.99       133
       Tamil       1.00      0.99      0.99        87
     Turkish       1.00      0.98      0.99       105

    accuracy              

## Benchmarking

In [19]:
def sample():
    sample_size = 1
    for i in range(0, len(x_test)-sample_size, sample_size):
        yield x_test[i:i+sample_size]

def benchmark_onnx():
    for t in sample():
        pred_onx = session.run(None, {"string_input": np.array([t]).reshape(len(t), 1)})

def benchmark_sklearn():
    for t in sample():
        pred = clf_pipeline.predict(np.array(t))

In [20]:
%time benchmark_sklearn()

CPU times: user 9.46 s, sys: 113 ms, total: 9.57 s
Wall time: 9.67 s


In [21]:
%time benchmark_onnx()

CPU times: user 6.07 s, sys: 53.9 ms, total: 6.13 s
Wall time: 3.1 s
